# H2O를 활용한 캘리포니아 집값 예측

이 노트북에서는 **H2O** 라이브러리를 사용하여 캘리포니아 집값 데이터를 전처리하고 예측 모델을 만드는 과정을 처음 사용하는 분들도 이해할 수 있도록 자세히 설명합니다.

---

## H2O란?

**H2O**는 오픈소스 머신러닝 플랫폼으로, 다음과 같은 장점이 있습니다:

- **대용량 데이터 처리**: 메모리 효율적인 분산 처리로 대용량 데이터를 빠르게 처리
- **AutoML 지원**: 자동으로 최적의 모델을 찾아주는 AutoML 기능 제공
- **다양한 알고리즘**: GBM, XGBoost, Random Forest, Deep Learning 등 다양한 알고리즘 지원
- **간편한 사용**: Python, R, Java 등 다양한 언어로 쉽게 사용 가능

---

## 1. H2O 설치 및 초기화

### 1.1 H2O 설치

H2O가 설치되어 있지 않다면 아래 명령어로 설치합니다.

In [ ]:
# H2O 설치 (최초 1회만 실행)
# !pip install h2o

### 1.2 라이브러리 불러오기

In [ ]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# H2O 라이브러리
import h2o
from h2o.automl import H2OAutoML

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

print(f"H2O 버전: {h2o.__version__}")

### 1.3 H2O 클러스터 초기화

H2O는 자체 클러스터(서버)를 실행하여 작동합니다. `h2o.init()`을 호출하면 로컬에서 H2O 클러스터가 시작됩니다.

**주요 파라미터:**
- `max_mem_size`: 사용할 최대 메모리 (예: "4G"는 4GB)
- `nthreads`: 사용할 CPU 코어 수 (-1은 모든 코어 사용)

In [ ]:
# H2O 클러스터 초기화
# max_mem_size: 사용할 최대 메모리 설정
# nthreads: 사용할 CPU 코어 수 (-1은 모든 코어 사용)
h2o.init(max_mem_size="4G", nthreads=-1)

# 클러스터 정보 확인
h2o.cluster().show_status()

---

## 2. 데이터 불러오기

H2O는 자체 데이터프레임 형식인 **H2OFrame**을 사용합니다. 
Pandas DataFrame과 비슷하지만 분산 처리에 최적화되어 있습니다.

### 2.1 CSV 파일 불러오기

In [ ]:
# H2O로 CSV 파일 직접 불러오기
# h2o.import_file()은 H2OFrame을 반환합니다
train_h2o = h2o.import_file("data/california-house-prices/train.csv")
test_h2o = h2o.import_file("data/california-house-prices/test.csv")

print(f"훈련 데이터 크기: {train_h2o.shape}")
print(f"테스트 데이터 크기: {test_h2o.shape}")

### 2.2 데이터 미리보기

H2OFrame도 Pandas처럼 `head()`, `describe()` 등의 메서드를 지원합니다.

In [ ]:
# 데이터 상위 5개 행 확인
train_h2o.head(5)

In [ ]:
# 컬럼 목록 확인
print("컬럼 목록:")
print(train_h2o.columns)

In [ ]:
# 각 컬럼의 데이터 타입 확인
print("컬럼별 데이터 타입:")
print(train_h2o.types)

### 2.3 기초 통계 확인

In [ ]:
# 수치형 컬럼 기초 통계
train_h2o.describe()

---

## 3. 데이터 탐색 (EDA)

H2OFrame을 Pandas DataFrame으로 변환하여 시각화할 수 있습니다.

### 3.1 타겟 변수 분포 확인

In [ ]:
# H2OFrame -> Pandas DataFrame 변환 (일부 데이터만)
# 대용량 데이터의 경우 전체를 변환하면 메모리 문제가 발생할 수 있음
train_pandas = train_h2o.as_data_frame()

# 타겟 변수(Sold Price) 분포 확인
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(train_pandas['Sold Price'].dropna(), bins=50, edgecolor='black')
plt.xlabel('Sold Price')
plt.ylabel('Frequency')
plt.title('집값 분포 (원본)')

plt.subplot(1, 2, 2)
plt.hist(np.log1p(train_pandas['Sold Price'].dropna()), bins=50, edgecolor='black')
plt.xlabel('Log(Sold Price)')
plt.ylabel('Frequency')
plt.title('집값 분포 (로그 변환)')

plt.tight_layout()
plt.show()

In [ ]:
# 타겟 변수 기초 통계
print("Sold Price 기초 통계:")
print(train_pandas['Sold Price'].describe())

### 3.2 결측치 확인

In [ ]:
# 각 컬럼의 결측치 비율 계산
missing_ratio = (train_pandas.isnull().sum() / len(train_pandas) * 100).sort_values(ascending=False)
print("결측치 비율 (상위 15개):")
print(missing_ratio.head(15).round(2))

In [ ]:
# 결측치 시각화
plt.figure(figsize=(12, 6))
missing_top = missing_ratio[missing_ratio > 0].head(20)
plt.barh(range(len(missing_top)), missing_top.values)
plt.yticks(range(len(missing_top)), missing_top.index)
plt.xlabel('결측치 비율 (%)')
plt.title('컬럼별 결측치 비율')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

---

## 4. 데이터 전처리

H2O는 기본적인 전처리를 자동으로 처리해주지만, 더 나은 성능을 위해 수동으로 전처리를 수행할 수 있습니다.

### 4.1 불필요한 컬럼 제거

In [ ]:
# 모델링에 사용하지 않을 컬럼 정의
# - Id: 식별자 (예측에 무의미)
# - Address: 주소 텍스트 (그대로 사용 어려움)
# - Summary: 설명 텍스트
# - Listed On, Last Sold On: 날짜 정보 (별도 처리 필요)

cols_to_remove = ['Id', 'Address', 'Summary', 'Listed On', 'Last Sold On']

# 컬럼 제거
# H2O에서는 drop() 대신 컬럼 선택으로 처리
cols_to_keep = [col for col in train_h2o.columns if col not in cols_to_remove]
train_h2o_clean = train_h2o[cols_to_keep]

# 테스트 데이터도 동일하게 처리 (Sold Price는 없음)
test_cols_to_keep = [col for col in cols_to_keep if col != 'Sold Price']
test_h2o_clean = test_h2o[test_cols_to_keep]

print(f"처리 후 훈련 데이터 컬럼 수: {len(train_h2o_clean.columns)}")
print(f"처리 후 테스트 데이터 컬럼 수: {len(test_h2o_clean.columns)}")

### 4.2 타겟 변수와 피처 분리

In [ ]:
# 타겟 변수 정의
target = 'Sold Price'

# 피처 컬럼 정의 (타겟 변수 제외)
features = [col for col in train_h2o_clean.columns if col != target]

print(f"타겟 변수: {target}")
print(f"피처 개수: {len(features)}")
print(f"\n피처 목록:")
for i, feat in enumerate(features, 1):
    print(f"  {i:2d}. {feat}")

### 4.3 데이터 타입 변환

H2O는 데이터를 불러올 때 자동으로 타입을 추론하지만, 필요시 수동으로 변환할 수 있습니다.

In [ ]:
# 현재 데이터 타입 확인
print("현재 데이터 타입:")
for col, dtype in train_h2o_clean.types.items():
    print(f"  {col}: {dtype}")

In [ ]:
# 범주형으로 변환할 컬럼들
categorical_cols = ['Type', 'Heating', 'Cooling', 'Parking', 'Bedrooms', 
                    'Region', 'City', 'Zip', 'State', 'Flooring',
                    'Heating features', 'Cooling features', 
                    'Appliances included', 'Laundry features', 'Parking features']

# 범주형으로 변환
for col in categorical_cols:
    if col in train_h2o_clean.columns:
        train_h2o_clean[col] = train_h2o_clean[col].asfactor()
    if col in test_h2o_clean.columns:
        test_h2o_clean[col] = test_h2o_clean[col].asfactor()

print("범주형 변환 완료!")

### 4.4 결측치 처리

H2O는 대부분의 알고리즘에서 결측치를 자동으로 처리합니다. 
그러나 필요한 경우 수동으로 처리할 수도 있습니다.

In [ ]:
# H2O에서 결측치 확인
print("각 컬럼의 결측치 개수:")
for col in train_h2o_clean.columns:
    na_count = train_h2o_clean[col].isna().sum()
    if na_count > 0:
        print(f"  {col}: {na_count}")

In [ ]:
# H2O의 결측치 처리 방법 예시
# 1. 수치형: 중앙값으로 대체
# 2. 범주형: 최빈값으로 대체

# H2O의 impute() 함수 사용
# method: "mean", "median", "mode" 선택 가능

# 예시: 특정 수치형 컬럼의 결측치를 중앙값으로 대체
# train_h2o_clean = train_h2o_clean.impute("Lot", method="median")

# 참고: H2O의 GBM, XGBoost 등은 결측치를 자체적으로 처리하므로
# 별도의 결측치 처리 없이도 모델링이 가능합니다.

print("H2O는 대부분의 알고리즘에서 결측치를 자동 처리합니다!")

### 4.5 훈련/검증 데이터 분할

In [ ]:
# H2O의 split_frame() 함수로 데이터 분할
# ratios: 분할 비율 (예: [0.8]이면 80% 훈련, 20% 검증)
# seed: 재현성을 위한 시드값

train_split, valid_split = train_h2o_clean.split_frame(
    ratios=[0.8],  # 80% 훈련, 20% 검증
    seed=42        # 재현성을 위한 시드
)

print(f"훈련 세트 크기: {train_split.shape}")
print(f"검증 세트 크기: {valid_split.shape}")

---

## 5. 모델 학습

H2O에서는 다양한 방법으로 모델을 학습할 수 있습니다:
1. **개별 모델**: GBM, XGBoost, Random Forest, Deep Learning 등
2. **AutoML**: 자동으로 여러 모델을 학습하고 최적 모델 선택

### 5.1 GBM (Gradient Boosting Machine) 모델

In [ ]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

# GBM 모델 정의
gbm_model = H2OGradientBoostingEstimator(
    model_id="gbm_california_house",  # 모델 ID
    ntrees=100,                        # 트리 개수
    max_depth=6,                       # 트리 최대 깊이
    learn_rate=0.1,                    # 학습률
    sample_rate=0.8,                   # 행 샘플링 비율
    col_sample_rate=0.8,               # 열 샘플링 비율
    seed=42,                           # 재현성 시드
    stopping_rounds=5,                 # 조기 종료 라운드
    stopping_metric="rmse",            # 조기 종료 기준 메트릭
    stopping_tolerance=0.001           # 조기 종료 허용 오차
)

# 모델 학습
gbm_model.train(
    x=features,           # 피처 컬럼
    y=target,             # 타겟 컬럼
    training_frame=train_split,   # 훈련 데이터
    validation_frame=valid_split  # 검증 데이터
)

print("GBM 모델 학습 완료!")

In [ ]:
# GBM 모델 성능 확인
print("=" * 50)
print("GBM 모델 성능")
print("=" * 50)

# 훈련 성능
train_perf = gbm_model.model_performance(train_split)
print(f"\n훈련 세트:")
print(f"  RMSE: {train_perf.rmse():,.2f}")
print(f"  MAE:  {train_perf.mae():,.2f}")
print(f"  R2:   {train_perf.r2():.4f}")

# 검증 성능
valid_perf = gbm_model.model_performance(valid_split)
print(f"\n검증 세트:")
print(f"  RMSE: {valid_perf.rmse():,.2f}")
print(f"  MAE:  {valid_perf.mae():,.2f}")
print(f"  R2:   {valid_perf.r2():.4f}")

### 5.2 Random Forest 모델

In [ ]:
from h2o.estimators.random_forest import H2ORandomForestEstimator

# Random Forest 모델 정의
rf_model = H2ORandomForestEstimator(
    model_id="rf_california_house",   # 모델 ID
    ntrees=100,                        # 트리 개수
    max_depth=10,                      # 트리 최대 깊이
    sample_rate=0.8,                   # 행 샘플링 비율
    seed=42,                           # 재현성 시드
    stopping_rounds=5,                 # 조기 종료 라운드
    stopping_metric="rmse"             # 조기 종료 기준 메트릭
)

# 모델 학습
rf_model.train(
    x=features,
    y=target,
    training_frame=train_split,
    validation_frame=valid_split
)

# 성능 확인
rf_valid_perf = rf_model.model_performance(valid_split)
print(f"\nRandom Forest 검증 성능:")
print(f"  RMSE: {rf_valid_perf.rmse():,.2f}")
print(f"  R2:   {rf_valid_perf.r2():.4f}")

### 5.3 XGBoost 모델

In [ ]:
from h2o.estimators.xgboost import H2OXGBoostEstimator

# XGBoost 모델 정의
xgb_model = H2OXGBoostEstimator(
    model_id="xgb_california_house",  # 모델 ID
    ntrees=100,                        # 트리 개수
    max_depth=6,                       # 트리 최대 깊이
    learn_rate=0.1,                    # 학습률
    sample_rate=0.8,                   # 행 샘플링 비율
    col_sample_rate=0.8,               # 열 샘플링 비율
    seed=42,                           # 재현성 시드
    stopping_rounds=5,                 # 조기 종료 라운드
    stopping_metric="rmse"             # 조기 종료 기준 메트릭
)

# 모델 학습
xgb_model.train(
    x=features,
    y=target,
    training_frame=train_split,
    validation_frame=valid_split
)

# 성능 확인
xgb_valid_perf = xgb_model.model_performance(valid_split)
print(f"\nXGBoost 검증 성능:")
print(f"  RMSE: {xgb_valid_perf.rmse():,.2f}")
print(f"  R2:   {xgb_valid_perf.r2():.4f}")

### 5.4 AutoML (자동 머신러닝)

**AutoML**은 H2O의 핵심 기능 중 하나로, 자동으로 여러 모델을 학습하고 최적의 모델을 찾아줍니다.

**AutoML이 수행하는 작업:**
- 다양한 알고리즘 학습 (GBM, XGBoost, Random Forest, Deep Learning 등)
- 하이퍼파라미터 튜닝
- 스태킹 앙상블 모델 생성
- 최적 모델 선택

In [ ]:
# AutoML 설정 및 학습
aml = H2OAutoML(
    max_models=10,           # 최대 학습할 모델 수
    max_runtime_secs=300,    # 최대 실행 시간 (초) - 5분
    seed=42,                 # 재현성 시드
    sort_metric="RMSE",      # 모델 정렬 기준
    verbosity="info"         # 로그 상세도
)

# AutoML 실행
print("AutoML 학습 시작... (최대 5분 소요)")
aml.train(
    x=features,
    y=target,
    training_frame=train_split,
    validation_frame=valid_split
)

print("\nAutoML 학습 완료!")

In [ ]:
# AutoML 리더보드 확인
# 리더보드는 성능 순으로 정렬된 모델 목록입니다
print("AutoML 리더보드:")
lb = aml.leaderboard
lb.head(10)

In [ ]:
# 최고 성능 모델 확인
best_model = aml.leader
print(f"\n최고 성능 모델: {best_model.model_id}")
print(f"모델 타입: {type(best_model).__name__}")

# 최고 모델 성능
best_perf = best_model.model_performance(valid_split)
print(f"\n검증 세트 성능:")
print(f"  RMSE: {best_perf.rmse():,.2f}")
print(f"  MAE:  {best_perf.mae():,.2f}")
print(f"  R2:   {best_perf.r2():.4f}")

---

## 6. 모델 해석

### 6.1 변수 중요도 (Feature Importance)

In [ ]:
# GBM 모델의 변수 중요도
print("GBM 모델 - 변수 중요도 (상위 15개):")
gbm_varimp = gbm_model.varimp(use_pandas=True)
print(gbm_varimp.head(15))

In [ ]:
# 변수 중요도 시각화
plt.figure(figsize=(10, 8))
top_features = gbm_varimp.head(15)
plt.barh(range(len(top_features)), top_features['scaled_importance'])
plt.yticks(range(len(top_features)), top_features['variable'])
plt.xlabel('Scaled Importance')
plt.title('GBM 모델 - 변수 중요도 (상위 15개)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

### 6.2 모델 성능 비교

In [ ]:
# 모든 모델의 검증 성능 비교
models = {
    'GBM': gbm_model,
    'Random Forest': rf_model,
    'XGBoost': xgb_model,
    'AutoML Best': best_model
}

print("모델 성능 비교 (검증 세트):")
print("=" * 60)
print(f"{'모델':<20} {'RMSE':>15} {'MAE':>15} {'R2':>10}")
print("=" * 60)

results = []
for name, model in models.items():
    perf = model.model_performance(valid_split)
    print(f"{name:<20} {perf.rmse():>15,.2f} {perf.mae():>15,.2f} {perf.r2():>10.4f}")
    results.append({'Model': name, 'RMSE': perf.rmse(), 'MAE': perf.mae(), 'R2': perf.r2()})

print("=" * 60)

In [ ]:
# 성능 비교 시각화
results_df = pd.DataFrame(results)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# RMSE 비교
axes[0].bar(results_df['Model'], results_df['RMSE'], color='steelblue')
axes[0].set_ylabel('RMSE')
axes[0].set_title('모델별 RMSE (낮을수록 좋음)')
axes[0].tick_params(axis='x', rotation=45)

# R2 비교
axes[1].bar(results_df['Model'], results_df['R2'], color='coral')
axes[1].set_ylabel('R2')
axes[1].set_title('모델별 R2 (높을수록 좋음)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

---

## 7. 예측 및 결과 저장

### 7.1 테스트 데이터 예측

In [ ]:
# 최고 성능 모델로 테스트 데이터 예측
# predict()는 H2OFrame을 반환합니다
predictions = best_model.predict(test_h2o_clean)

# 예측 결과 확인
print("예측 결과 (상위 10개):")
predictions.head(10)

In [ ]:
# 예측 결과를 Pandas DataFrame으로 변환
predictions_df = predictions.as_data_frame()

# 예측 결과 통계
print("예측 결과 통계:")
print(predictions_df['predict'].describe())

### 7.2 제출 파일 생성

In [ ]:
# 테스트 데이터의 Id 가져오기
test_ids = test_h2o['Id'].as_data_frame()

# 제출 파일 생성
submission = pd.DataFrame({
    'Id': test_ids['Id'],
    'Sold Price': predictions_df['predict']
})

# 제출 파일 확인
print("제출 파일 미리보기:")
print(submission.head(10))

# CSV 파일로 저장
submission.to_csv('submission_h2o.csv', index=False)
print(f"\n제출 파일 저장 완료: submission_h2o.csv")
print(f"파일 크기: {len(submission)} 행")

### 7.3 모델 저장 및 불러오기

In [ ]:
# 모델 저장
# H2O 모델은 바이너리 형식으로 저장됩니다
model_path = h2o.save_model(model=best_model, path="./models", force=True)
print(f"모델 저장 경로: {model_path}")

In [ ]:
# 저장된 모델 불러오기 (예시)
# loaded_model = h2o.load_model(model_path)
# loaded_predictions = loaded_model.predict(test_h2o_clean)

print("모델 저장/불러오기 완료!")

---

## 8. H2O 클러스터 종료

작업이 완료되면 H2O 클러스터를 종료하여 리소스를 해제합니다.

In [ ]:
# H2O 클러스터 종료
# 주의: 종료 후에는 H2O 관련 작업을 수행할 수 없습니다
# h2o.shutdown(prompt=False)

print("H2O 클러스터 종료는 위 명령어의 주석을 해제하여 실행하세요.")

---

## 9. 요약 및 팁

### H2O 사용 핵심 정리

| 단계 | H2O 함수/메서드 | 설명 |
|------|----------------|------|
| 초기화 | `h2o.init()` | H2O 클러스터 시작 |
| 데이터 로드 | `h2o.import_file()` | CSV 파일을 H2OFrame으로 로드 |
| 데이터 분할 | `df.split_frame()` | 훈련/검증 데이터 분할 |
| 모델 학습 | `model.train()` | 모델 학습 |
| 예측 | `model.predict()` | 새 데이터 예측 |
| 성능 평가 | `model.model_performance()` | 모델 성능 확인 |
| AutoML | `H2OAutoML()` | 자동 머신러닝 |
| 모델 저장 | `h2o.save_model()` | 모델 저장 |
| 종료 | `h2o.shutdown()` | H2O 클러스터 종료 |

### 추가 팁

1. **메모리 관리**: 대용량 데이터 작업 시 `max_mem_size`를 적절히 설정
2. **조기 종료**: `stopping_rounds` 파라미터로 과적합 방지
3. **교차 검증**: `nfolds` 파라미터로 k-fold 교차 검증 수행 가능
4. **앙상블**: AutoML의 스태킹 앙상블 모델 활용
5. **MOJO 내보내기**: 프로덕션 배포 시 `download_mojo()` 사용

---

## 10. 참고 자료

- [H2O 공식 문서](https://docs.h2o.ai/)
- [H2O Python API](https://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/index.html)
- [H2O AutoML 가이드](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html)
- [H2O GitHub](https://github.com/h2oai/h2o-3)